In [ ]:
from langchain_core.runnables import RunnableConfig
from langchain.schema import AIMessage
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from pydantic import BaseModel, Field
from fpdf import FPDF
import random
import pdfplumber
import warnings
import dotenv

warnings.filterwarnings("ignore")
dotenv.load_dotenv()

In [ ]:
class State(TypedDict):
    query : Annotated[str, "User Question"]
    answer : Annotated[str, "LLM response"]
    messages : Annotated[list, add_messages]
    tool_call : Annotated[dict, "Tool Call Result"]

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini",
                 temperature=0.,)

In [ ]:
@tool
def read_pdf(file_path: str) -> str:
    """
    PDF 파일에서 텍스트를 추출하는 도구입니다.
    표 형식 또는 일반 텍스트가 포함된 PDF를 읽고 문자열로 반환합니다.
    
    file_path 예시: './files/report.pdf'
    """
    try:
        text = ""
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        return text.strip() if text.strip() else "❌ PDF에서 텍스트를 추출할 수 없습니다."
    except Exception as e:
        return f"❌ PDF 읽기 오류: {str(e)}"

In [ ]:
@tool
def write_pdf(content: str, filename: str = "output.pdf"):
    """
    텍스트를 PDF 파일로 저장하는 도구입니다.
    PDF형태의 문서로 만들어야할 때 이 도구를 사용하세요.
    """
    
    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)

    font_path = "C:\Windows\Fonts\MALGUN.TTF"  # <-- 여기에 실제 폰트 파일이 있어야 함

    try:
        pdf.add_font("malgun", "", font_path, uni=True)
        pdf.set_font("malgun", size=12)
    except:
        raise ValueError("한글 폰트가 존재하지 않습니다.")
    
    for line in content.split("\n"):
        pdf.multi_cell(0, 10, line)
    pdf.output(f"./files/{filename}")

    print(f"PDF saved as ./files/{filename}")

    return {"content":content, "filename":filename}

In [ ]:
tools = [TavilySearchResults(include_domains=["naver.com", "google.com"]), PythonAstREPLTool(), write_pdf, read_pdf, *FileManagementToolkit(root_dir="./files/",
                                                                            selected_tools=["file_delete","list_directory"]).get_tools()]
search_tool, code_tool, write_tool, read_tool, delete_tool, listdir_tool= tools

In [ ]:
llm_with_tools = llm.bind_tools(tools)

In [ ]:
class HistoryChecker(BaseModel):
    """
    이전의 대화 기록을 참고하여 질문에 대해 답변할 수 있는지 판단합니다.
    답변할 수 있다면 "yes", 답변할 수 없다면 "no"를 반환합니다.
    """

    yes_no : Literal["yes", "no"] = Field(..., description="""Use your previous conversation history to determine if you can answer your questions.
    Return "yes" if you can answer, "no" if you can't answer.""")

In [ ]:
def shorterm_memory(state:State):

    if len(state["messages"]) > 6:
        history = state["messages"][-6:-1]
    elif len(state["messages"]) == 1:
        history = ""
    else:
        history = state["messages"][:-1]

    return history

In [ ]:
def history_check(state:State):

    prompt = PromptTemplate.from_template("""

                이전의 대화 기록을 참고하여 질문에 대해 답변할 수 있는지 판단합니다.
                답변할 수 있다면 "yes", 답변할 수 없다면 "no"를 반환합니다.
                
                대화 기록 : {history}
                                          
                질문 : {query}
                                          
                """)
    
    chain = prompt | history_checker

    history = shorterm_memory(state)

    result = chain.invoke({"history":history,
                            "query":state["query"]})

    return result.yes_no

In [ ]:
def memory_chat(state:State):

    prompt = PromptTemplate.from_template("""

                이전의 대화 기록을 참고하여 질문에 대해 답변하세요.
                아래 대화 기록을 첨부합니다.
                대화 기록을 통해 답변이 어렵다면 내부 지식을 참조하세요.
                
                대화 기록 : {history}
                                          
                질문 : {query}
                                          
                """)

    
    chain = prompt | llm

    history = shorterm_memory(state)

    answer = chain.invoke({"history":history,
                           "query":state["query"]})
    
    if len(state["tool_call"]) == 0:
        return {"answer":answer.content,
                "messages":[answer],
                "tool_call":"사용된 기록 없음."}
    else:
        return {"answer":answer.content,
                "messages":[answer]}

In [ ]:
history_checker = llm.with_structured_output(HistoryChecker)

In [ ]:
def history_node(state:State):
    if len(state["messages"]) == 1:
        return {"answer":"답변 없음",
                "tool_call":"사용된 도구 없음"}
    else:
        return state

In [ ]:
def select(
    state: State,
):

    prompt = PromptTemplate.from_template("""

                이전의 대화 기록을 참고하여 질문에 대해 답변하세요.
                아래 대화 기록을 첨부합니다.
                대화 기록을 통해 답변이 어렵다면 내부 지식을 참조하세요.
                최근에 사용한 도구가 있다면 도구도 참고하세요. 다른 도구를 사용하는 것이 더 좋은 방법이 될 수 있습니다.
                                                    
                대화 기록 : {history}     
                                          
                최근 사용한 도구 : {tool_name}
                                        
                정답 : {answer}
                                        
                질문 : {query}
                                          
                """)

    chain = prompt | llm_with_tools

    history = shorterm_memory(state)

    result = chain.invoke({"history" : history,
                           "tool_name" : state["tool_call"],
                            "answer": state["answer"],
                            "query": state["query"]})

    if hasattr(result, "tool_calls") and len(result.tool_calls) > 0:
        tool_calls = result.tool_calls
        return {"messages": result,
                "tool_call": [result]}
    else:
        return {"messages":AIMessage(content=f"""도구를 선택하지 못했습니다. 적절한 도구를 재선택하세요.
                                        """),
                                    "tool_call":None}

    

In [ ]:
tool_node = ToolNode(tools)

In [ ]:
class AnswerChecker(BaseModel):
    """
    정답 분류기입니다.
    
    정답이 질문을 해결했는지 여부를 판단합니다.
    질문을 해결하지 못했을 시 해결될 때까지 도구를 이용합니다.

    질문을 해결했다면 "end", 해결하지 못했다면 "tool"을 반환합니다.
    """


    end : Literal["end", "tool"] = Field(..., description="""You are the answer sorter.

                                                                Determine if the correct answer has solved the question.
                                                                If the question is not resolved, use the tool until it is resolved.

                                                                Return "end" if you solved the question, or "tool" if you didn't.""")    

In [ ]:
answer_checker = llm.with_structured_output(AnswerChecker)

In [ ]:
def response(state:State):

    return {"answer":state["messages"][-1]}

In [ ]:
def answer_check(state:State):

    prompt = PromptTemplate.from_template("""
    당신은 정답 분류기 어시스턴트입니다.
    
    정답이 질문을 해결할 수 있는지 여부를 판단합니다.
    질문을 해결할 수 없다면 도구를 이용합니다.

    질문을 해결할 수 있다면 "end", 아니라면 "tool"을 반환합니다.
                                          
    기존 History도 참고하여 답변하세요.
                                          
    History : {history}
                            
    정답 : {answer}
                            
    질문 : {query}
    """)

    chain = prompt | answer_checker

    history = shorterm_memory(state)

    result = chain.invoke({"history" : history,
                            "answer": state["answer"],
                            "query": state["query"]})
    
    return result.end

In [ ]:
graph_builder = StateGraph(State)

In [ ]:
graph_builder.add_node("history_node", history_node)
graph_builder.add_node("memory_chat", memory_chat)
graph_builder.add_node("select", select)
graph_builder.add_node("tools", tool_node)
graph_builder.add_node("response", response)


graph_builder.add_edge(START, "history_node")
graph_builder.add_conditional_edges("history_node",
                            history_check,
                            {"yes":"memory_chat",
                             "no":"select"})
graph_builder.add_edge("select", "tools")
graph_builder.add_edge("tools", "response")
graph_builder.add_edge("memory_chat", "response")
graph_builder.add_conditional_edges("response",
                                    answer_check,
                                    {"end":END,
                                    "tool":"select"});

In [ ]:
memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

In [ ]:
def reset_config(limit=20):

    thread_id=random.randint(1,999999)

    config = RunnableConfig(recursion_limit=limit, configurable={"thread_id": thread_id})

    return config

In [ ]:
config = reset_config()

In [ ]:
graph

In [ ]:
def streaming(query, config):

    result = graph.stream({"messages":("user", query),
                         "query":query}, config=config)
    for step in result:
        for k, v in step.items():
            print(f"\n\n=== {k} ===\n\n")
            print(v)
    
    return 

In [ ]:
config = reset_config()

query = "1+1은 뭔가요?"

streaming(query, config)

In [ ]:
query = "테슬라에 대해 조사해서 레포트 작성해서 pdf형태로 저장해주세요."

streaming(query, config)

In [ ]:
query = "방금 조사한 tesla_report.pdf파일 한글로 번역해서 다시 써줘"

streaming(query, config)

In [ ]:
query = "방금 번역한 내용을 테슬라_레포트.pdf파일로 저장해줘. 루트는 './files/'에 저장해주면 돼."

streaming(query, config)

In [ ]:
config = reset_config()

query = "'tesla_report.pdf' 삭제해줘"

streaming(query, config)

In [ ]:
config = reset_config()

streaming("pda1.pdf파일 읽어줘", config=config)

In [ ]:
query = "개강하는 날짜는 언제인가요?"

streaming(query, config)

In [ ]:
query = "강의는 어디서 들을 수 있나요?"

streaming(query, config)

In [ ]:
config = reset_config()

code = """
아래 코드 실행시켜주세요.

```python

result = 0

for i in range(20):
    print(f"{i+1}번째 출력: ", i)
    result += i

print("최종 결과: ", result)

```
"""

streaming(code, config)

In [ ]:
config = reset_config()

streaming("""모두의연구소는 어떤 곳이야?
          깔끔하게 정리해서 레포트로 만들어줘.
          레포트의 형식은 pdf로 저장해주면 돼.
          이름은 "모두의연구소_레포트.pdf"로 해줘.""", config)